In [26]:
%load_ext autoreload
%autoreload 2

import logging
import requests
import json
import os
import time
import pandas as pd
import numpy as np

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

from collections import deque
from datetime import datetime, timedelta, date
from logging.handlers import TimedRotatingFileHandler
from fred.api import Fred
from fred.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect
from fred.upload import *

now = datetime.now()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
conf = get_jinja_yaml_conf('./conf/api.yml', './conf/logging.yml')
end_date = datetime.now().date() 


# logger 설정
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                when=conf['logging']['when'],
                                interval=conf['logging']['interval'],
                                backupCount=conf['logging']['backupCount']), logging.StreamHandler()]
                )


In [28]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

In [29]:
engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
fred = Fred(auth_key = os.environ['auth_key'])

2024-04-08 15:43:22,036 (utils.py 54) INFO ::: Connect to 172.20.10.3. DB_NAME is economics
2024-04-08 15:43:22,037 (api.py 23) INFO ::: ### fred main api is initialized! ###


# 1. id

In [71]:
data = pd.DataFrame(fred.get_category(0))
stack = deque([0])

while stack:
    res = fred.get_category_children(category_id = stack.popleft())

    if res.shape[0] == 0:
        continue

    stack += res['id'].tolist()
    data = postgres_conn.ext_notin_db(res, schema_name = conf['schema_name'], table_name = conf['data']['id']['table_name'], subset = 'id')
    postgres_conn.insert_db(data, schema_name = conf['schema_name'], table_name = conf['data']['id']['table_name'])
    



2024-02-20 00:49:04,088 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:04,597 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:04,960 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:05,247 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:05,626 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:05,962 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:06,292 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:06,609 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:06,975 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:07,258 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:07,584 (utils.py 124) INFO ::: Upload data successfully (rows: 0).
2024-02-20 00:49:07,877 (utils.py 124) INFO ::: Upload data successfully (ro

KeyboardInterrupt: 

# 2. tags

In [90]:
fred.get_api_data?

Signature: fred.get_api_data(detail_url='', sub_url='', params={}, max_retries=30)
Docstring: <no docstring>
File:      /projects/data_collection/economics/fred/fred/api.py
Type:      method

In [119]:
a = fred.get_api_data(detail_url='series', sub_url='updates', params = {'realtime_start': '2000-01-01', 'offset': 50808}, max_retries=3)

In [129]:
a = fred.get_api_data(detail_url='tags', sub_url='series', params = {'realtime_start': '2000-01-01', 'tag_names': 'food', 'offset': 32000}, max_retries=3)

In [130]:
a

{'realtime_start': '2000-01-01',
 'realtime_end': '9999-12-31',
 'order_by': 'series_id',
 'sort_order': 'asc',
 'count': 30440,
 'offset': 32000,
 'limit': 1000,
 'seriess': []}

# 3. Releases

# 4. Series

In [7]:
id_list = postgres_conn.get_data(schema_name = conf['schema_name'], table_name = conf['data']['releases']['table_name'], columns = 'id', is_distinct = True)
id_list = id_list.values.ravel()


for idx, release_id in enumerate(id_list):
    if release_id != 462:
        continue
    logger.info(f"upload series related release_id {release_id} processing... ({idx}/{len(id_list)})")
    max_ts = postgres_conn.get_maxmin_col(conf['schema_name'], conf['data']['series']['table_name'], column = '_ts',
                                where = [f"release_id={release_id}"])[0]

    max_ts = (max_ts - timedelta(days = 3)).strftime('%Y-%m-%d') if max_ts else conf['earliest_date']
    

    data = pd.DataFrame()
    db_last_updated = postgres_conn.get_maxmin_col(conf['schema_name'], conf['data']['series']['table_name'], column = 'last_updated',
                                    where = [f"release_id={release_id}"])[0]
    offset, count = 0, 1000
            
    while offset < count:
        
        res = fred.get_api_data(detail_url = 'release', sub_url = 'series', 
                            params = {'realtime_start': max_ts, 'release_id': release_id,
                                     'order_by': 'last_updated', 'sort_order': 'desc', 'offset': offset})
    
        count = res['count'] if 'count' in res else len(res)
        offset += res['limit']
    
        data = pd.concat([data, pd.DataFrame(res['seriess'])])
        
        logger.info(f"series data load status: {min(offset, count)} / {count}.")
    
        min_last_update = pd.to_datetime(data['last_updated'], utc = True).min() if data.shape[0] > 0 else pd.to_datetime(conf['earliest_date'], utc = True)
    
    
        if db_last_updated is not None and db_last_updated > min_last_update:
            logger.info("lastest data load finished. Start to upload task.")
            break

    if data.shape[0] == 0:
        continue

    data['_ts'] = os.environ['_ts']
    
    data = data.sort_values('last_updated').drop_duplicates(conf['data']['series']['dup_cols'])
    data['release_id'] = release_id
    data = data[pd.to_datetime(data['last_updated'], utc = True) >= db_last_updated]
    postgres_conn.upsert(data, conf['schema_name'], conf['data']['series']['table_name'])


logger.info('Upload series finished!')

2024-04-08 00:39:52,229 (1801020834.py 8) INFO ::: upload series related release_id 462 processing... (280/317)
2024-04-08 00:40:03,425 (1801020834.py 31) INFO ::: series data load status: 1000 / 193377.


2342


2024-04-08 00:40:14,486 (1801020834.py 31) INFO ::: series data load status: 2000 / 193377.


2342


KeyboardInterrupt: 

In [36]:
data = fred.get_data(detail_url = 'release', sub_url = 'series', 
                            params = {'realtime_start': max_ts, 'release_id': release_id})

2024-04-08 15:46:13,502 (api.py 100) INFO ::: release/series data, parameters {'realtime_start': datetime.date(1776, 7, 4), 'release_id': 462, 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 1000} load status: 1000 / 193377.
2024-04-08 15:46:13,813 (api.py 100) INFO ::: release/series data, parameters {'realtime_start': datetime.date(1776, 7, 4), 'release_id': 462, 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 2000} load status: 2000 / 193377.
2024-04-08 15:46:14,149 (api.py 100) INFO ::: release/series data, parameters {'realtime_start': datetime.date(1776, 7, 4), 'release_id': 462, 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 3000} load status: 3000 / 193377.
2024-04-08 15:46:14,447 (api.py 100) INFO ::: release/series data, parameters {'realtime_start': datetime.date(1776, 7, 4), 'release_id': 462, 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 4000} load sta

In [37]:
data['release_id'] = release_id
postgres_conn.upsert(data, conf['schema_name'], conf['data']['series']['table_name'])

2024-04-08 15:56:03,890 (utils.py 171) INFO ::: Upload data starts: 193377 rows.


377

# 5. observations

In [7]:
data = fred.get_data(detail_url= 'series', sub_url = 'observations', params = {'series_id': 'PENLISCOUYY13089', 'output_type': 1})

In [63]:
fred.get_data(detail_url= 'series', sub_url = 'vintagedates', params = {'series_id': 'DMPCRATE049023'})

,0,_ts
0,2017-02-03,2024-02-24 22:23:00.032038+09:00
1,2018-07-03,2024-02-24 22:23:00.032038+09:00


In [ ]:
upload_observations(postgres_conn, fred, conf)

2024-04-08 16:57:42,724 (api.py 58) WARNING ::: 404 Error code: There is no data in series, params: {'series_id': 'GDPFDEFGBFP', 'sort_order': 'desc', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json'}
2024-04-08 16:57:42,725 (upload.py 128) ERROR ::: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=GDPFDEFGBFP&sort_order=desc&api_key=60c2303c4d56f2bdf6349c588df1943c&file_type=json
2024-04-08 16:57:43,000 (api.py 58) WARNING ::: 404 Error code: There is no data in series, params: {'series_id': 'GNPFDEFGBFP', 'sort_order': 'desc', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json'}
2024-04-08 16:57:43,000 (upload.py 128) ERROR ::: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=GNPFDEFGBFP&sort_order=desc&api_key=60c2303c4d56f2bdf6349c588df1943c&file_type=json
2024-04-08 16:57:43,341 (api.py 99) INFO ::: series/observations data, parameters {'series_i

In [59]:
obs_latest_date = postgres_conn.get_data(conf['schema_name'], conf['data']['observations']['table_name'], columns = ['id', 'max(date) AS obs_max_date'],
                      additional_sql = 'GROUP BY id')

series_latest_date = postgres_conn.get_data(conf['schema_name'], conf['data']['series']['table_name'], columns = ['id', 'max(observation_end) AS series_max_date'],
                               additional_sql = 'GROUP BY id')

date_info = series_latest_date.merge(obs_latest_date, how = 'left', on = 'id')

id_list = date_info.loc[date_info.apply(lambda x: True if x['obs_max_date'] is np.nan else x['series_max_date'] > x['obs_max_date'], axis = 1), 'id'].ravel()

for idx, series_id in enumerate(id_list):


    data = fred.get_data(detail_url= 'series', sub_url = 'observations', params = {'series_id': series_id}).drop_duplicates(keep = 'last')
    data['id'] = series_id
    data['value'] = data['value'].replace('.', np.nan)
    # data = postgres_conn.ext_notin_db(data, conf['schema_name'], conf['data']['observations']['table_name'], subset = conf['data']['observations']['dup_cols'])
    postgres_conn.upsert(data, conf['schema_name'], conf['data']['observations']['table_name'])
    logger.info(f"Upload {series_id} data finished! ({idx+1}/{len(id_list)})")

2024-04-06 14:58:46,655 (api.py 90) INFO ::: series/observations data, parameters {'series_id': 'CEU3232900001', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 1000} load status: 411 / 411.
2024-04-06 14:58:46,694 (utils.py 171) INFO ::: Upload data starts: 411 rows.
2024-04-06 14:58:46,703 (3397365832.py 31) INFO ::: Upload CEU3232900001 data finished! (1/79843)
2024-04-06 14:58:47,083 (api.py 90) INFO ::: series/observations data, parameters {'series_id': 'H8B3094NFRD', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 1000} load status: 1000 / 2674.
2024-04-06 14:58:47,485 (api.py 90) INFO ::: series/observations data, parameters {'series_id': 'H8B3094NFRD', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 2000} load status: 2000 / 2674.
2024-04-06 14:58:47,779 (api.py 90) INFO ::: series/observations data, parameters {'series_id': 'H8B3094NFRD', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'fi

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "observations_pkey"
DETAIL:  Key (id, date)=(H8B3094NFRD, 1992-03-04) already exists.

[SQL: INSERT INTO fred.observations (realtime_start, realtime_end, date, value, _ts, id) VALUES (%(realtime_start__0)s, %(realtime_end__0)s, %(date__0)s, %(value__0)s, %(_ts__0)s, %(id__0)s), (%(realtime_start__1)s, %(realtime_end__1)s, %(date__1)s, %(valu ... 107078 characters truncated ... ltime_start__999)s, %(realtime_end__999)s, %(date__999)s, %(value__999)s, %(_ts__999)s, %(id__999)s)]
[parameters: {'realtime_end__0': '2024-04-06', 'id__0': 'H8B3094NFRD', '_ts__0': datetime.datetime(2024, 4, 6, 12, 31, 37, 903793, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'KST')), 'realtime_start__0': '2024-04-06', 'value__0': '561964.0', 'date__0': '2011-05-04', 'realtime_end__1': '2024-04-06', 'id__1': 'H8B3094NFRD', '_ts__1': datetime.datetime(2024, 4, 6, 12, 31, 37, 903793, tzinfo=datetime.time

# 6. dates

In [6]:
db_first_realdate = postgres_conn.get_maxmin_col(conf['schema_name'], conf['data']['releases']['table_name'], column = 'realtime_start',
                                )[1]

db_latest_date = postgres_conn.get_maxmin_col(conf['schema_name'], conf['data']['releases_date']['table_name'], column = 'date',
                                )[0]

realtime_start = max(db_first_realdate, db_latest_date) if db_latest_date else db_first_realdate

data = fred.get_data('releases', 'dates', params = {'realtime_start': db_first_realdate,
                                                'include_release_dates_with_no_data': 'true'})

postgres_conn.upsert(data, conf['schema_name'], conf['data']['releases_date']['table_name'])
logger.info(f"Upload releases_date data finished!")

2024-04-06 12:31:43,212 (api.py 60) INFO ::: API call tries 1/30
2024-04-06 12:31:43,213 (api.py 61) INFO ::: {"error_code":429,"error_message":"Too Many Requests.  Exceeded Rate Limit"}
2024-04-06 12:31:48,470 (api.py 60) INFO ::: API call tries 2/30
2024-04-06 12:31:48,470 (api.py 61) INFO ::: {"error_code":429,"error_message":"Too Many Requests.  Exceeded Rate Limit"}
2024-04-06 12:31:53,721 (api.py 60) INFO ::: API call tries 3/30
2024-04-06 12:31:53,721 (api.py 61) INFO ::: {"error_code":429,"error_message":"Too Many Requests.  Exceeded Rate Limit"}
2024-04-06 12:31:58,993 (api.py 60) INFO ::: API call tries 4/30
2024-04-06 12:31:58,993 (api.py 61) INFO ::: {"error_code":429,"error_message":"Too Many Requests.  Exceeded Rate Limit"}
2024-04-06 12:32:11,061 (api.py 90) INFO ::: releases/dates data, parameters {'realtime_start': datetime.date(1927, 1, 26), 'include_release_dates_with_no_data': 'true', 'api_key': '60c2303c4d56f2bdf6349c588df1943c', 'file_type': 'json', 'offset': 1000

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "release_last_updated" of relation "releases_date" violates not-null constraint
DETAIL:  Failing row contains (2024-04-06 03:31:37.903793+00, 472, Daily Federal Funds Rate, null, 2020-06-26).

[SQL: INSERT INTO fred.releases_date (release_id, release_name, release_last_updated, date, _ts) VALUES (%(release_id__0)s, %(release_name__0)s, %(release_last_updated__0)s, %(date__0)s, %(_ts__0)s), (%(release_id__1)s, %(release_name__1)s, %(release_last_ ... 105196 characters truncated ... elease_id__999)s, %(release_name__999)s, %(release_last_updated__999)s, %(date__999)s, %(_ts__999)s)]
[parameters: {'release_name__0': 'Interest on Required Balances and Excess Balances', 'release_id__0': 185, '_ts__0': datetime.datetime(2024, 4, 6, 12, 31, 37, 903793, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400), 'KST')), 'release_last_updated__0': '2024-04-05 15:34:54-05', 'date__0': '2020-07-13', 'release_name__1': 'Inter

In [7]:
postgres_conn.upsert(data, conf['schema_name'], conf['data']['releases_date']['table_name'])

2024-04-06 12:56:26,913 (utils.py 171) INFO ::: Upload data starts: 106660 rows.


660